In [2]:
import os
import sys

import polars as pl
import torch


'''
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128
https://developer.nvidia.com/cuda-12-8-0-download-archive
'''

MAC_DIR = '/Users/igwanhyeong/PycharmProjects/data_research/raw_data/'
WINDOW_DIR = 'C:/Users/USER/PycharmProjects/research/raw_data/'

if sys.platform == 'win32':
    DIR = WINDOW_DIR
    print(torch.cuda.is_available())
    print(torch.cuda.device_count())
    print(torch.version.cuda)
    print(torch.__version__)
    print(torch.cuda.get_device_name(0))
    print(torch.__version__)
else:
    DIR = MAC_DIR
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

save_dir = DIR + 'fit/20251115_LTB'

In [2]:
target_dyn_demand_monthly = pl.read_parquet(DIR + 'target_dyn_demand_monthly.parquet').sort(['oper_part_no', 'demand_dt'])
target_dyn_demand_monthly = (target_dyn_demand_monthly
                                .group_by('oper_part_no', maintain_order = True)
                                .map_groups(lambda g: g.with_columns(pl.arange(1, len(g) + 1).alias('sequence')))
                            )


filtered_target = (target_dyn_demand_monthly
                    .group_by('oper_part_no')
                    .agg(pl.col('sequence').max().alias('sequence_max'))
                    .filter(pl.col('sequence_max') > 43)
                    .select('oper_part_no')
                   ) # seq Q75

target_dyn_demand_monthly = (target_dyn_demand_monthly
                                .join(filtered_target, on = 'oper_part_no', how = 'right')
                                .select(['oper_part_no', 'demand_dt', 'sequence', 'demand_qty'])
                             )
target_dyn_demand_monthly


oper_part_no,demand_dt,sequence,demand_qty
str,i64,i64,f64
"""01023-50612""",201801,1,10.0
"""01023-50612""",201804,2,2.0
"""01023-50612""",201805,3,20.0
"""01023-50612""",201808,4,50.0
"""01023-50612""",201809,5,101.0
…,…,…,…
"""Y173-A0020""",202607,83,665.0
"""Y173-A0020""",202609,84,1205.0
"""Y173-A0020""",202612,85,3150.0


In [7]:
# target_dyn_demand = pl.read_parquet(DIR + 'parquets/dyn_demand.parquet').drop('part_no')

(target_dyn_demand_monthly
    .sort(['oper_part_no', 'sequence'])
    .with_columns(pl.col('demand_qty')
    .rolling_mean(window_size = 12, min_samples = 1).over('oper_part_no').alias('demand_qty'))
    .with_columns(pl.col('demand_qty'))
 )

oper_part_no,demand_dt,sequence,demand_qty
str,i64,i64,f64
"""01023-50612""",201801,1,10.0
"""01023-50612""",201804,2,6.0
"""01023-50612""",201805,3,10.666667
"""01023-50612""",201808,4,20.5
"""01023-50612""",201809,5,36.6
…,…,…,…
"""Y173-A0020""",202607,83,1969.583333
"""Y173-A0020""",202609,84,2065.0
"""Y173-A0020""",202612,85,2065.0


In [4]:
from resources.util.preprocess_util.demand_resampler import DemandResampler

resampler = DemandResampler(
    target_dyn_demand,
    name_col="oper_part_no",
    date_col="demand_dt",
    target_col="demand_qty",
    date_fmt="%Y%m%d",
)

# 1) 주간: Date + iso_yyyyww
weekly_df = resampler.to_weekly_filled(as_int=False, add_iso_yyyyww=True)
weekly_df

oper_part_no,demand_dt,demand_qty,iso_yyyyww
str,date,f64,i64
"""0001-1001""",2018-03-12,5.0,201811
"""0001-1001""",2018-03-19,0.0,201812
"""0001-1001""",2018-03-26,0.0,201813
"""0001-1001""",2018-04-02,0.0,201814
"""0001-1001""",2018-04-09,0.0,201815
…,…,…,…
"""ZZ90239""",2023-05-22,0.0,202321
"""ZZ90239""",2023-05-29,0.0,202322
"""ZZ90239""",2023-06-05,0.0,202323


In [5]:
from resources.util.preprocess_util.intermittent_demand_detector import IntermittentConfig, IntermittentDemandDetector

cfg = IntermittentConfig(
    name_col="oper_part_no",
    target_col="demand_qty",
    adi_threshold=1.32,
    cv2_threshold=0.49,
    count_threshold=20,   # 20 미만은 사실상 0으로 취급
    use_cv2=True,
    min_periods=20,
)

detector = IntermittentDemandDetector(weekly_df, config=cfg)

# 1) 간헐 여부 플래그만
sparsity_flag = detector.detect(return_stats=False)
# → oper_part_no | is_sparsity

# 2) 수요 유형(smooth / erratic / intermittent / lumpy / insufficient)
demand_type = detector.classify(return_stats=False)
# → oper_part_no | demand_type

# 3) 타입 + ADI / CV²까지 같이 보고 싶을 때
demand_type_detail = detector.classify(return_stats=True)
# → oper_part_no | demand_type | is_sparsity | n_periods | ... | ADI | CV2

In [6]:
demand_type_detail

oper_part_no,demand_type,is_sparsity,n_periods,n_zero,n_nz,zero_ratio,ADI,CV2
str,str,bool,u32,u32,u32,f64,f64,f64
"""T2305-25341""","""insufficient""",false,1,1,0,1.0,null,null
"""V025-401""","""insufficient""",false,178,178,0,1.0,null,null
"""M301210""","""insufficient""",false,285,285,0,1.0,null,null
"""P6270-83152""","""intermittent""",true,316,254,62,0.803797,5.096774,0.188961
"""T2185-21243""","""insufficient""",false,318,318,0,1.0,null,null
…,…,…,…,…,…,…,…,…
"""3688C018""","""insufficient""",false,195,195,0,1.0,null,null
"""T5855-21401""","""insufficient""",false,299,299,0,1.0,null,null
"""T4681-82982OE""","""insufficient""",false,1,1,0,1.0,null,null


/Users/igwanhyeong/PycharmProjects/data_research/modeling_module/model_test
